**Sinergia Dev** playbook

# Openai Responses API

- python install requirements

In [27]:
!pip install openai

In [28]:
import getpass
import os

# set env for openai key:

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [32]:
from openai import OpenAI
client = OpenAI()

# Generate text from a simple prompt

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o-mini",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio."
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

En un bosque encantado bajo un cielo estrellado, un unicornio de brillante cuerno dorado danzaba entre las flores, llenando el aire de suaves susurros mágicos que arrullaban a todos los animales hasta que se quedaban dormidos, soñando con aventuras en mundos lejanos.
response object:

{'created_at': 1742693934.0,
 'error': None,
 'id': 'resp_67df662e9fe48192a9e673150b83a2c205de7011f3d2c940',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': 'En un bosque encantado bajo un cielo '
                                  'estrellado, un unicornio de brillante '
                                  'cuerno dorado danzaba entre las flores, '
                                  'llenando el aire de suaves susurros mágicos '
                                  'que arrullaban a todos los animales hasta '
                  

In [34]:
# Generate text with instructions

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o",
    instructions="Your goals is to translate to english every user input",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio.",
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

response.output_text:

Once upon a time, a unicorn with a shimmering rainbow mane gently guided the stars across the night sky to help lull the world to sleep.
response object:

{'created_at': 1742694376.0,
 'error': None,
 'id': 'resp_67df67e81f248192bc34a81eb67c65f80527f92665842bd4',
 'incomplete_details': None,
 'instructions': 'Your goals is to translate to english every user input',
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-2024-08-06',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': 'Once upon a time, a unicorn with a '
                                  'shimmering rainbow mane gently guided the '
                                  'stars across the night sky to help lull the '
                                  'world to sleep.',
                          'type': 'output_text'}],
             'id': 'msg_67df67e86fbc8192a59ebf9eb925f7410527f92665842bd4',
             'role': 'assistant',
             'status

In [38]:
# Generate text with messages using different roles

## ChatGPT LLM with message history style (it will preserve instructions on history)
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "developer",
            "content": "Your goals is to translate to spanish every user input"
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

response.output_text:

¿Son opcionales los puntos y coma en JavaScript?
response object:

{'created_at': 1742696160.0,
 'error': None,
 'id': 'resp_67df6ee020e0819287dd0e3440d2334801e4bf2825cab03a',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': '¿Son opcionales los puntos y coma en '
                                  'JavaScript?',
                          'type': 'output_text'}],
             'id': 'msg_67df6ee092808192bf4409fa423b50fb01e4bf2825cab03a',
             'role': 'assistant',
             'status': 'completed',
             'type': 'message'}],
 'parallel_tool_calls': True,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None},
 'status': 'completed',
 'store': True,
 'temperature': 1.0,
 'text': {'format': {'type': 'text'}},
 'tool_choice': 'auto',
 'to

# Structured Outputs

In [41]:
import json

# structured output models support by:

# gpt-4.5-preview-2025-02-27 and later
# o3-mini-2025-1-31 and later
# o1-2024-12-17 and later
# gpt-4o-mini-2024-07-18 and later
# gpt-4o-2024-08-06 and later

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Matias and Jony are going to SinergIA-Dev on 25 Friday, march at 15hs."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string"
                    },
                    "date": {
                        "type": "string"
                    },
                    "participants": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

event = json.loads(response.output_text)

print(event)

{'name': 'SinergIA-Dev', 'date': '2023-03-25T15:00:00', 'participants': ['Matias', 'Jony']}


In [44]:
import json
import pprint

# Structured Outputs for chain-of-thought math tutoring

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": { "type": "string" },
                                "output": { "type": "string" }
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": { "type": "string" }
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

math_reasoning = json.loads(response.output_text)
pprint.pprint(math_reasoning)


{'final_answer': 'x = -\\frac{15}{4}',
 'steps': [{'explanation': "Start by isolating the term with the variable 'x'. "
                           'We need to move the constant term on the left side '
                           'to the right side. To do this, subtract 7 from '
                           'both sides of the equation.',
            'output': '8x + 7 - 7 = -23 - 7'},
           {'explanation': 'This simplifies to:', 'output': '8x = -30'},
           {'explanation': "Next, solve for 'x' by dividing both sides of the "
                           'equation by 8.',
            'output': '\\( x = \\frac{-30}{8} \\)'},
           {'explanation': 'Simplify the fraction by dividing the numerator '
                           'and the denominator by their greatest common '
                           'divisor, which is 2.',
            'output': '\\( x = \\frac{-15}{4} \\)'},
           {'explanation': 'Thus, the solution to the equation is:',
            'output': 'x = -\\frac{15}{4

In [6]:
# install langchain

!pip install langchain
!pip install -qU "langchain[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.6 MB/s eta 0:00:00


In [26]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

# define model to use
model = init_chat_model("gpt-4o-mini", model_provider="openai")

# message preparation:

## using history messages:
messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi! Im a developer trying to learn AI"),
]

# LLM call

## invoke chat completion
response = model.invoke(messages)

# print response text
print("response.content:\n")
print(response.content + "\n")
print("response object:\n")
pprint.pprint(response.model_dump())


response.content:

¡Hola! Soy un desarrollador que está tratando de aprender sobre IA.

response object:

{'additional_kwargs': {'refusal': None},
 'content': '¡Hola! Soy un desarrollador que está tratando de aprender sobre '
            'IA.',
 'example': False,
 'id': 'run-6d050664-d16b-4c6b-827f-547a56ab17a3-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': 'chatcmpl-BE4TgDzmLvW32p12Wq7bFpTJMRnSu',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'system_fingerprint': 'fp_b8bc95a0ac',
                       'token_usage': {'completion_tokens': 16,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 0,
          

In [ ]:
# example for a dynamic template completion

## using template with variables
system_template = "Translate the following from English into {language}" # variable on template interpolated with {var_name}

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({"language": "Italian", "text": "hi! Please take the recommended actions to update your configuration"})



In [13]:
template_prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = template_prompt | model
response1 = chain.invoke({"topic": "programming"})
print(response1.content)

Why do programmers prefer dark mode?  

Because light attracts bugs!


In [15]:
import pprint

pprint.pprint(response1.model_dump())


{'additional_kwargs': {'refusal': None},
 'content': 'Why do programmers prefer dark mode?  \n'
            '\n'
            'Because light attracts bugs!',
 'example': False,
 'id': 'run-f082ff71-da76-4769-bdbe-6d1b237dd666-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': 'chatcmpl-BE3qElWGd9rZe1lS6y28RiISSkMhz',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'system_fingerprint': 'fp_b8bc95a0ac',
                       'token_usage': {'completion_tokens': 14,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 0,
                                                                     'rejected_prediction_tokens':

**Sinergia Dev** playbook

# Openai Responses API

- python install requirements

In [ ]:
!pip install openai

In [ ]:
import getpass
import os

# set env for openai key:

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [ ]:
from openai import OpenAI
client = OpenAI()

# Generate text from a simple prompt

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o-mini",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio."
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

En un bosque encantado bajo un cielo estrellado, un unicornio de brillante cuerno dorado danzaba entre las flores, llenando el aire de suaves susurros mágicos que arrullaban a todos los animales hasta que se quedaban dormidos, soñando con aventuras en mundos lejanos.
response object:

{'created_at': 1742693934.0,
 'error': None,
 'id': 'resp_67df662e9fe48192a9e673150b83a2c205de7011f3d2c940',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': 'En un bosque encantado bajo un cielo '
                                  'estrellado, un unicornio de brillante '
                                  'cuerno dorado danzaba entre las flores, '
                                  'llenando el aire de suaves susurros mágicos '
                                  'que arrullaban a todos los animales hasta '
                  

In [ ]:
# Generate text with instructions

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o",
    instructions="Your goals is to translate to english every user input",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio.",
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

response.output_text:

Once upon a time, a unicorn with a shimmering rainbow mane gently guided the stars across the night sky to help lull the world to sleep.
response object:

{'created_at': 1742694376.0,
 'error': None,
 'id': 'resp_67df67e81f248192bc34a81eb67c65f80527f92665842bd4',
 'incomplete_details': None,
 'instructions': 'Your goals is to translate to english every user input',
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-2024-08-06',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': 'Once upon a time, a unicorn with a '
                                  'shimmering rainbow mane gently guided the '
                                  'stars across the night sky to help lull the '
                                  'world to sleep.',
                          'type': 'output_text'}],
             'id': 'msg_67df67e86fbc8192a59ebf9eb925f7410527f92665842bd4',
             'role': 'assistant',
             'status

In [ ]:
# Generate text with messages using different roles

## ChatGPT LLM with message history style (it will preserve instructions on history)
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "developer",
            "content": "Your goals is to translate to spanish every user input"
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

response.output_text:

¿Son opcionales los puntos y coma en JavaScript?
response object:

{'created_at': 1742696160.0,
 'error': None,
 'id': 'resp_67df6ee020e0819287dd0e3440d2334801e4bf2825cab03a',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'text': '¿Son opcionales los puntos y coma en '
                                  'JavaScript?',
                          'type': 'output_text'}],
             'id': 'msg_67df6ee092808192bf4409fa423b50fb01e4bf2825cab03a',
             'role': 'assistant',
             'status': 'completed',
             'type': 'message'}],
 'parallel_tool_calls': True,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None},
 'status': 'completed',
 'store': True,
 'temperature': 1.0,
 'text': {'format': {'type': 'text'}},
 'tool_choice': 'auto',
 'to

# Structured Outputs

In [ ]:
import json

# structured output models support by:

# gpt-4.5-preview-2025-02-27 and later
# o3-mini-2025-1-31 and later
# o1-2024-12-17 and later
# gpt-4o-mini-2024-07-18 and later
# gpt-4o-2024-08-06 and later

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Matias and Jony are going to SinergIA-Dev on 25 Friday, march at 15hs."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string"
                    },
                    "date": {
                        "type": "string"
                    },
                    "participants": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

event = json.loads(response.output_text)

print(event)

{'name': 'SinergIA-Dev', 'date': '2023-03-25T15:00:00', 'participants': ['Matias', 'Jony']}


In [ ]:
import json
import pprint

# Structured Outputs for chain-of-thought math tutoring

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": { "type": "string" },
                                "output": { "type": "string" }
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": { "type": "string" }
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

math_reasoning = json.loads(response.output_text)
pprint.pprint(math_reasoning)


{'final_answer': 'x = -\\frac{15}{4}',
 'steps': [{'explanation': "Start by isolating the term with the variable 'x'. "
                           'We need to move the constant term on the left side '
                           'to the right side. To do this, subtract 7 from '
                           'both sides of the equation.',
            'output': '8x + 7 - 7 = -23 - 7'},
           {'explanation': 'This simplifies to:', 'output': '8x = -30'},
           {'explanation': "Next, solve for 'x' by dividing both sides of the "
                           'equation by 8.',
            'output': '\\( x = \\frac{-30}{8} \\)'},
           {'explanation': 'Simplify the fraction by dividing the numerator '
                           'and the denominator by their greatest common '
                           'divisor, which is 2.',
            'output': '\\( x = \\frac{-15}{4} \\)'},
           {'explanation': 'Thus, the solution to the equation is:',
            'output': 'x = -\\frac{15}{4

In [ ]:
# install langchain

!pip install langchain
!pip install -qU "langchain[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.6 MB/s eta 0:00:00


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

# define model to use
model = init_chat_model("gpt-4o-mini", model_provider="openai")

# message preparation:

## using history messages:
messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi! Im a developer trying to learn AI"),
]

# LLM call

## invoke chat completion
response = model.invoke(messages)

# print response text
print("response.content:\n")
print(response.content + "\n")
print("response object:\n")
pprint.pprint(response.model_dump())


response.content:

¡Hola! Soy un desarrollador que está tratando de aprender sobre IA.

response object:

{'additional_kwargs': {'refusal': None},
 'content': '¡Hola! Soy un desarrollador que está tratando de aprender sobre '
            'IA.',
 'example': False,
 'id': 'run-6d050664-d16b-4c6b-827f-547a56ab17a3-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': 'chatcmpl-BE4TgDzmLvW32p12Wq7bFpTJMRnSu',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'system_fingerprint': 'fp_b8bc95a0ac',
                       'token_usage': {'completion_tokens': 16,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 0,
          

In [ ]:
# example for a dynamic template completion

## using template with variables
system_template = "Translate the following from English into {language}" # variable on template interpolated with {var_name}

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({"language": "Italian", "text": "hi! Please take the recommended actions to update your configuration"})



In [ ]:
template_prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = template_prompt | model
response1 = chain.invoke({"topic": "programming"})
print(response1.content)

Why do programmers prefer dark mode?  

Because light attracts bugs!


In [ ]:
import pprint

pprint.pprint(response1.model_dump())


{'additional_kwargs': {'refusal': None},
 'content': 'Why do programmers prefer dark mode?  \n'
            '\n'
            'Because light attracts bugs!',
 'example': False,
 'id': 'run-f082ff71-da76-4769-bdbe-6d1b237dd666-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': 'chatcmpl-BE3qElWGd9rZe1lS6y28RiISSkMhz',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'system_fingerprint': 'fp_b8bc95a0ac',
                       'token_usage': {'completion_tokens': 14,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 0,
                                                                     'rejected_prediction_tokens':